# Graph Building
This notebook contains the code that implements the graph building procedure.
Starting from the dictionary of matches found in the Feature Matching phase, it is possible to build a graph whose nodes are the images and edges are the pairwise homographies. This graph is useful both for visualization and for optimization.
Here we also compute the matrices needed for the various stitching methods (Z, C, M) and we perform graph expansion to deal with multi-edges graphs.

## Importing libraries

In [220]:
import cv2 as cv
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import networkx as nx
from IPython.display import Audio, display
import ipynb.fs.defs.Utils as Utils

## Functions definition

In [ ]:
# Utility function, for testing purposes
def allDone():
    print(" --- finished --- ")
    #display(Audio(url='https://sound.peal.io/ps/audios/000/001/131/original/kon.wav', autoplay=True))

### Graph Expansion
If the graph has multi-edges, i.e. multiple links between two nodes (i, j), we follow the procedure described by [DalCin2021](https://doi.org/10.1109/ICCV48922.2021.00639) to build a new expanded graph that do not have multi-edges.

In [ ]:
# Returns: The adjacency matrix of the expanded graph (adj_matrix_exp), the replicas structure (replicas_structure)
def expand_graph(adj_matrix # Adjacency matrix of the multi-graph
                ):
    
    # Dict that associates to each node in the multi-graph his replicas present in the expanded graph.
    replicas_structure = dict()
    
    adj_matrix_exp = np.copy(adj_matrix)
    
    # Check if there are multi-edges that need to be expanded
    while np.sum(adj_matrix_exp > 1) > 0:
        
        # Consider only the number of replicated edges
        adj_matrix_multi = np.maximum(adj_matrix_exp - 1, 0)
        
        # The number of outgoing edges for each node
        edges_out = np.sum(adj_matrix_multi, axis=1)
        
        # Take the node with the maximum number of out-going multi-edges, this will be tho node to replicate
        node_max = np.argmax(edges_out)
        
        # The number of replicas for the node is equal to the maximum cardinality of its outgoing multi-edges
        replicas = np.max(adj_matrix_multi[node_max,:])
        
        # Append one column for each replica of the node, keeping the same ingoing arcs as the original node
        appended_column = np.copy(adj_matrix_exp[:,node_max])
        appended_column[node_max]=1
        for i in range(replicas):
            adj_matrix_exp = np.c_[adj_matrix_exp,appended_column]

        # Store the number of multi-edges of the original node, use them as counters
        row = np.copy(adj_matrix_exp[node_max,:])
        
        # Change the cardinality of the outgoing edges for the original node, all the cardinalities > 1 are set to 1, since the multi-edges have been expanded 
        appended_row = np.minimum(1,row)
        adj_matrix_exp[node_max,:] = appended_row
        
        # Update the number of multi-edges that needs to be linked to the new replicas
        row = np.maximum(0, row-1)
        
        # For each replica, assign the remaining links
        for i in range(replicas):
            appended_row = np.minimum(1,row)
            adj_matrix_exp = np.r_[adj_matrix_exp, [appended_row]]
            row = np.maximum(0,row-1)
        
        # The replicas of the node will be: the node itself, and the indexes of the new columns of the adjacency matrix
        replicas_structure[node_max] = [node_max] +  list(range(adj_matrix_exp.shape[0] - replicas, adj_matrix_exp.shape[0]))
    
    # For the nodes that have not been expanded, we add them to the replicas structure, having only themselves as replicas
    for i in range(adj_matrix.shape[0]):
        if i not in replicas_structure:
            replicas_structure[i] = [i]
            
    return adj_matrix_exp, replicas_structure

### Matrices computation
We compute the matrices needed for the stitching methods and for the synchronization procedure

In [ ]:
# Returns: the Z matrix, a 3N x 3N matrix containing the pairwise homographies. N is the number of nodes.
def compute_Z_matrix(n_expanded,  # number of nodes of the expanded graph
                     replicas,    # replicas structure
                     matches_dict # Dictionary of matches between images
                    ):
    # Initialize Z as the identity. In this way each matrix along the principal diagonal is an I(3), 
    # correct since it corresponds to an homography from a node to itself.
    Z = np.eye(3*n_expanded, 3*n_expanded)
    
    # For each match, defined by the nodes (i,j), assign the list of associated homographies "matches_dict[(i, j)]" to the correct place in the Z matrix. 
    for couple in matches_dict:
        i,j = couple
        
        # iterate over the replicas of the source node
        for k in range(len(replicas[i])):
            # not all the multi-edges starting from "i" have the maximum cardinality. 
            if (k >= len(matches_dict[couple])):
                break
                
            # consider a different homography for each replica
            h = matches_dict[couple][k]
           
            # assign "h" to all the edges starting from the current replica of "i" and going to a replica of "j"
            for w in replicas[j]:
                Z[3*w:3*(w+1), 3*replicas[i][k]:3*(replicas[i][k]+1)] = h
    
    # Add identity constraints between the original node and each replica
    for replica in replicas:
        r = replicas[replica]
        for k in r:
            Z[3*k:3*(k+1), 3*r[0]:3*(r[0]+1)] = np.eye(3)

    return Z

In [ ]:
# Returns: the C matrix, a 3N x 3R matrix that specifies hard constraints for the replicated nodes.
def compute_C_matrix(n_expanded, # number of nodes of the expanded graph
                     n,          # number of nodes of the original graph
                     replicas    # replicas structure
                    ):
    C = np.zeros([3*n_expanded, 3*(n_expanded-n)], dtype=int)
    
    # for each replica we fill add a column that is seen as constraints between two nodes:
    # this column has a 3x3 matrix for each node, for the original node we use the I(3), for one of the replicas we use -I(3), for all other nodes zeroes(3).
    for replica in replicas:
        r = replicas[replica]
        first = r[0]
        for k in r:
            if k != first:
                C[3*first:3*(first+1),3*(k-n):3*(k-n+1)] = np.eye(3)
                C[3*k:3*(k+1),        3*(k-n):3*(k-n+1)] = -np.eye(3)

    return C

In [ ]:
# Returns: the M matrix, a 3N x 3N matrix that is based on Z and is needed for the synchronization procedure.
def compute_M_matrix(adj_matrix, # adjacency matrix
                     Z           # Z matrix 
                    ):
    # number of nodes
    n = adj_matrix.shape[0]
    
    # diagonal matrix containing the number of outgoing arcs for each node
    D = np.diag(np.sum(adj_matrix + np.eye(n), axis=1))
    # expansion of the D matrix
    D_kron = np.kron(D, np.eye(3))
    
    #compute M
    M = Z - D_kron
    return M

### Main functions
Functions called from other notebooks

In [ ]:
# From the matches dictionary, builds the expanded graph and computes the matrices
# Returns: the M, Z, C, adj_matrix_exp matrices
def build_multi_graph_matrices(dataset_name, # Name of the dataset to be used
                imgs,                        # Images to be stitched
                matches_dict,                # Dictionary of matches between images
                output_dir ="output",        # Name of the output directory
                verbose = True,              # If True prints additional graphs and images
                save_output = True           # If True saves the output of the procedure
               ):

    # Filenames for the matrices
    M_filename = f"M_{dataset_name}.npy"
    C_filename = f"C_{dataset_name}.npy"
    Z_filename = f"Z_{dataset_name}.npy"
    Adj_filename = f"Adj_{dataset_name}.npy"

    graph_dir = os.path.join(output_dir,'graph')

    if save_output:
        if not os.path.isdir(graph_dir):
            os.makedirs(graph_dir)

    # number of nodes
    n = len(imgs)

    # compute the adjacency matrix from the matches
    adj_matrix = np.zeros([n,n], dtype=int)
    for (i,j) in matches_dict:
        adj_matrix[i, j] = len(matches_dict[i,j])

    # expand the graph
    adj_matrix_exp, replicas = expand_graph(adj_matrix)

    if verbose:
        Utils.build_and_print_multi_graph(adj_matrix_exp, save_output, graph_dir, "graph")

    # number of nodes of the expanded graph
    n_expanded = adj_matrix_exp.shape[0]
    
    # compute the matrices
    Z = compute_Z_matrix(n_expanded, replicas, matches_dict)
    C = compute_C_matrix(n_expanded, n, replicas)
    M = compute_M_matrix(adj_matrix_exp, Z)

    if save_output:
        np.save(os.path.join(output_dir,M_filename), M)
        np.save(os.path.join(output_dir,Z_filename), Z)
        np.save(os.path.join(output_dir,C_filename), C)
        np.save(os.path.join(output_dir,Adj_filename), adj_matrix_exp)

    if verbose:
        allDone()

    return M, Z, C, adj_matrix_exp
        

In [ ]:
# From the matches dictionary, keeps only one match per images pair, builds the expanded graph and computes the matrices using the above function
# Returns: the M, Z, adj_matrix matrices
def build_simple_graph_matrices(dataset_name, # Name of the dataset to be used
                imgs,                        # Images to be stitched
                matches_dict,                # Dictionary of matches between images
                output_dir ="output",        # Name of the output directory
                verbose = True,              # If True prints additional graphs and images
                save_output = True           # If True saves the output of the procedure
               ):
    
    # Keep only the last match for each couple, in order to remove multi-edges
    simple_matches_dict = dict()
    for couple in matches_dict:
        simple_matches_dict[couple] = [matches_dict[couple][-1]]
    
    # use the above function, a simple-graph is a particular case of a multi-graph.
    M, Z, _, adj_matrix = build_multi_graph_matrices(
        dataset_name = dataset_name,
        imgs = imgs,
        matches_dict = simple_matches_dict,
        output_dir = output_dir,
        verbose = verbose,
        save_output = save_output
    )

    return M, Z, adj_matrix

In [ ]:
# From the matches dictionary, averages the matches between the same images pair, builds the expanded graph and computes the matrices using the above function
# Returns: the M, Z, adj_matrix matrices
def build_edge_averaging_matrices(dataset_name, # Name of the dataset to be used
                imgs,                        # Images to be stitched
                matches_dict,                # Dictionary of matches between images
                output_dir ="output",        # Name of the output directory
                verbose = True,              # If True prints additional graphs and images
                save_output = True           # If True saves the output of the procedure
               ):
    
    # builds a new dict with only one edge per couple, computed as the average of the multi-edges
    edge_averaging_dict = dict()
    for couple in matches_dict:
        stack = np.stack(matches_dict[couple])
        avg_edge = [np.mean(stack, axis = 0)]
        det = np.linalg.det(avg_edge)
        edge_averaging_dict[couple] = avg_edge/np.cbrt(det)
    
    # use the above function, a simple-graph is a particular case of a multi-graph.
    M, Z, _, adj_matrix = build_multi_graph_matrices(
        dataset_name = dataset_name,
        imgs = imgs,
        matches_dict = edge_averaging_dict,
        output_dir = output_dir,
        verbose = verbose,
        save_output = save_output
    )

    return M, Z, adj_matrix